# The data

In this script we will compare out 2 catalogs @kovlakasHeraklionExtragalacticCatalogue2021 and [@karachentsevUPDATEDNEARBYGALAXY2013, @karachentsevSTARFORMATIONPROPERTIES2013a]

-   The data have been joined based on their position in the sky (Ra, Dec, Distance).
-   We use TOPCAT to create two joins, an inner and an outer join
-   We will use the inner join for 1-1 comparisons
-   If we see that the data are similar we can use the outer join


In [ ]:
#| echo: false
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy.table import Table, join, QTable
import astropy.units as u
from astropy.visualization import quantity_support, hist
#quantity_support()
from astropy.stats import sigma_clip, SigmaClip
from astropy.modeling import models, fitting
from scipy.stats import pearsonr
import seaborn as sns
import pandas as pd
import glob
import os
plt.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.f' % x)

dt = QTable(ascii.read("../tables/inner_join.ecsv"), masked=True)

The dataset we are going to use consists of `{python} len(dt)` galaxies and `{python} len(dt.colnames)` columns.

# How are we going to compare the data?

## Scatter plots and $R^2$ calculation

1.  $R^2$: Measures the proportion of variance explained by the linear model.
2.  Slope of the Fitted Line: Should be close to 1 for a 1-1 correlation.[^1]
3.  Pearson Correlation $\rho$: Measures the strength and direction of the linear relationship between two variables, ranging from -1 to 1. [^2]

[^1]: Fitting can be done using the uncertainties as weights. To get the standard weighting of 1/unc\^2, assuming Gaussian errors, the weights to pass to the fitting are 1/unc.

[^2]: In simple linear regression, $R^2$ is the square of the Pearson correlation coefficient $\rho$.


In [ ]:
def compare_data(x, y, sigma = False):
    """
    Performs a linear comparison between two datasets.
    
    This function fits a linear model to the data, calculates the slope and intercept of the fitted line,
    and computes the R-squared value and Pearson correlation coefficient to assess the fit quality.
    
    Parameters:
    - x (array-like): The independent variable data.
    - y (array-like): The dependent variable data.
    - unc (array-like with units, optional): The uncertainties associated with the a variable data. Default is None.

    Returns:
    tuple: A tuple containing the following elements:
        - slope (float): The slope of the fitted linear model.
        - intercept (float): The intercept of the fitted linear model.
        - r2 (float): The R-squared value, indicating the proportion of variance explained by the linear model.
        - corr (float): The Pearson correlation coefficient, measuring the linear correlation between x and y.
    """
    try:
        x_data = np.ma.array(x.value, mask=np.isnan(x.value))
        y_data = np.ma.array(y.value, mask=np.isnan(y.value))
        # initialize a linear fitter
        fit = fitting.LinearLSQFitter()

        # initialize a linear model
        line_init = models.Linear1D()

        # fit the data with the fitter
        # check if sigma
        if sigma is False:
            fitted_line = fit(line_init, x_data, y_data)
            outliers = None
        else:
            or_fit = fitting.FittingWithOutlierRemoval(fit, sigma_clip, niter =3, sigma=3)
            fitted_line, outliers = or_fit(line_init, x_data, y_data)
        slope = fitted_line.slope.value
        intercept = fitted_line.intercept.value

        # Predict values using the fitted model
        y_pred = fitted_line(x_data)

        # Remove NaN values
        mask = ~np.isnan(y_data)
        if outliers is not None:
            mask = ~np.isnan(y_data) & ~outliers
        y_data_clean = y_data[mask]
        y_pred_clean = y_pred[mask]

        # Calculate R-squared
        r2 = r2_score(y_data_clean, y_pred_clean)

        # Calculate Pearson correlation coefficient
        corr = np.sqrt(np.abs(r2))

        return slope, intercept, r2, corr, outliers
    except Exception:
        return 0,0,0,0,None

4.  [Plots]{.underline}: Plots are essential for visually assessing the relationship between two datasets, identifying correlations, trends, and outliers, and evaluating the fit of linear models.


In [ ]:
def scatter_plot(x, y, xerr = None, yerr = None, sigma = False, logscale = False):
    """
    Generates a scatter plot of two datasets with optional error bars, fits a linear model to the data, 
    and displays the fitted line on the plot.

    Parameters:
    - x (array-like with units): Independent variable data.
    - y (array-like with units): Dependent variable data.
    - xerr (array-like, optional): Error in the independent variable data. Default is 0.
    - yerr (array-like, optional): Error in the dependent variable data. Default is 0.

    Returns:
    None. The function displays a scatter plot with a fitted linear model.
    """
    # Convert data to masked arrays, masking NaN values
    x_data = np.ma.array(x.value, mask=np.isnan(x.value))
    y_data =  np.ma.array(y.value, mask=np.isnan(y.value))

    # Handle default values for xerr and yerr
    if xerr is None:
        xerr_d = 0
    else:
        xerr_d = np.ma.array(xerr.value, mask=np.isnan(xerr))
    if yerr is None:
        yerr_d = 0
    else:
        yerr_d = np.ma.array(yerr.value, mask=np.isnan(yerr))

    
    # Perform linear comparison between the datasets
    c, m, r2, corr, outliers = compare_data(x, y, sigma)

    # Plot the data with error bars
    # errorbar -> needs np.ma -> no units
    if outliers is not None:
        filtered_data = np.ma.masked_array(y.value , mask = outliers)
        outlier_percentage = (np.sum(outliers) / len(outliers)) * 100

        plt.errorbar(x.value, y.value, yerr=yerr_d, c = "black", label="Clipped Data, {} ({}%)".format(np.sum(outliers), round(outlier_percentage,1)), fmt = "o", fillstyle = "none")
        plt.scatter(x[~outliers], y[~outliers],  label=f"Fitted Data, {np.sum(~filtered_data.mask)}", c = "blue")

    else:
        plt.errorbar(x_data, y_data, xerr = xerr_d, yerr = yerr_d, alpha = 0.3, c = "blue", label = "Data", fmt = ".")
    
    # Plot the fitted line 
    if c!=0:
        plt.plot(x, c*x.value+ m, c = "red", label = f"Fit: {y.info.name}$_H $= {c:.2f}$\cdot${x.info.name}$_L${m:+.2f}\n $R^2=$ {r2:.2f}")

    # Set plot labels
    plt.xlabel(f"LCV, {x.info.name} [{x.unit}]")
    plt.ylabel(f"HECATE, {y.info.name} [{y.unit}]")

    # Display legend
    plt.legend()
    
    # Show the plot
    plt.show()

Some data seem to have a very good linear correlation but they have many outliers. This is why we will clip the outliers with $sigma > 3$

-   Correlation Heatmaps: A correlation heatmap is a graphical tool that displays the correlation between multiple variables as a color-coded matrix. It’s like a color chart that shows us how closely related different variables are.

In a correlation heatmap, each variable is represented by a row and a column, and the cells show the correlation between them. The color of each cell represents the strength and direction of the correlation, with darker colors indicating stronger correlations.

-   Histograms: Because not all of our data have the same number of counts, the comparison with histograms between data that are not the same, doesn't help us right now.[^3] This is why we will only use histograms for comparing the distribution of same-data columns normalized by their maximum value

[^3]: When we will use the outer join table we could use histograms due to the large number of counts.

<!-- -->

-   Percentage change: We can calculate the percentage change of the data for each galaxy and then we can see if the data are similar, based on minimum, the maximum and the mean value of the difference.

$$\text{Percentage change} = \frac{V_{Hecate} - V_{LCV}}{V_{Hecate}}\cdot 100 \%$$


In [ ]:
def relative_diff(hec, lcv):
    """
    Calculate the relative difference between two sets of values.

    Parameters:
    hec (numpy.ndarray): An array of values representing the HEC dataset.
    lcv (numpy.ndarray): An array of values representing the LCV dataset.

    Returns:
    numpy.ndarray: An array of relative differences between the HEC and LCV datasets.
    The relative difference is calculated as ((HEC - LCV) * 100) / |HEC|.
    Any infinite values in the result are replaced with NaN.
    """
    diff = (hec - lcv) * 100 / np.abs(hec)
    diff[np.isinf(diff)] = np.nan
    return diff

def percent_desc_histogram(x, sigmaclip = True, zoom = None):
    """
    This function creates a histogram of the given data and optionally applies a 3-sigma clip.
    
    Parameters:
    - x (array-like): The input data to create the histogram.
    - sigmaclip (bool, optional): Whether to apply a 3-sigma clip to the data. Default is True.
    - zoom (tuple, optional): The range of values to display on the x-axis of the histogram. Default is None.
    
    Returns:
    - t_pd (DataFrame): A pandas DataFrame containing statistical information about the input data.
    
    The function uses the seaborn library to create the histogram and applies a 3-sigma clip if `sigmaclip` is True.
    The histogram is displayed with appropriate labels and titles.
    The statistical information about the input data is returned as a pandas DataFrame.
    """
    plt.close()
    temp_table = QTable()
    temp_table["Percentage change"]= x.copy()
    
    if sigmaclip is True:
        temp_table["Percentage change, after 3 sigma clip"] = sigma_clip(temp_table["Percentage change"],3)
    
    t_pd = temp_table.to_pandas()
    
    sns.histplot(temp_table["Percentage change"], kde = True, label = "Percentage change")
    
    if sigmaclip is True:
        sns.histplot(temp_table["Percentage change, after 3 sigma clip"], kde = True, label = "Percentage change, after 3 sigma clip")
    
    plt.xlim(zoom)
    plt.xlabel("Percentage change Distribution [%]")
    plt.title("Types of Galaxies")
    plt.legend()
    plt.show()
    
    return t_pd.describe(percentiles=[]).map('{:,.0f}'.format).style.set_properties(**{'text-align': 'center'})

# Comparable data

## Coordinates


In [ ]:
#| echo: false
ra_corr = round(compare_data(dt["Ra_1"],dt["RA_2"])[3],3)
dec_corr = round(compare_data(dt["Dec_1"],dt["DEC_2"])[3],3)
d_corr = round(compare_data(dt["Dis"],dt["D"])[3],3)

|  LCV  | HECATE |   Description   | Pearson Correlation \[-1,1\] |
|:-----:|:------:|:---------------:|:----------------------------:|
| Ra_1  |  RA_2  | Right Ascension |      `{python} ra_corr`      |
| Dec_1 | DEC_2  |   Declination   |     `{python} dec_corr`      |
|  Dis  |   D    |    Distance     |      `{python} d_corr`       |

::: panel-tabset
## Right Ascension


In [ ]:
scatter_plot(dt["Ra_1"], dt["RA_2"])

## Declination


In [ ]:
scatter_plot(dt["Dec_1"], dt["DEC_2"]), 

## Distance


In [ ]:
scatter_plot(dt["Dis"], dt["D"])

:::

## Velocities


In [ ]:
rvel_corr = round(compare_data(dt["RVel"],dt["V"])[3],3)
rvel_corr_v = round(compare_data(dt["RVel"],dt["V_VIR"])[3],3)
vlg_corr = round(compare_data(dt["VLG"],dt["V"])[3],3)
vlg_corr_v = round(compare_data(dt["VLG"],dt["V_VIR"])[3],3)
cz_corr = round(compare_data(dt["cz"],dt["V"])[3],3)
cz_corr_v = round(compare_data(dt["cz"],dt["V_VIR"])[3],3)

|     LCV     |    HECATE    |              Description               |  Linear Correlation  |
|:----------------:|:-----------------:|:----------------:|:----------------:|
| RVel (km/s) |   V (km/s)   |      Heliocentric radial velocity      | `{python} rvel_corr` |
| VLG (km/s)  |              |            Radial velocity             |                      |
|  cz (km/s)  |              |         Heliocentric velocity          |                      |
|             | V_VIR (km/s) | Virgo-infall corrected radial velocity |                      |

::: panel-tabset
## Heliocentric radial Velocity


In [ ]:
scatter_plot(dt["RVel"], dt["V"], yerr = dt["E_V"])

## Scatter Grid


In [ ]:
vel_data = dt[["RVel", "V", "VLG", "cz", "V_VIR"]].to_pandas()

# Create a pair grid
sns.pairplot(vel_data, corner=True)
plt.show()

\[?\] The close correlation between all of the velocities, could be due to the fact that all of them measure the velocity of each galaxy, but from a different frame of reference. 

## Heatmap


In [ ]:
# Compute the correlation matrix
corr_matrix = vel_data.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Heatmap of Velocity Values")
plt.show()

:::

## Morphology and Geometry


In [ ]:
dt["INCL"].mask = np.isnan(dt["INCL"])

In [ ]:
#| echo: false
 
ttype_corr = round(compare_data(dt["TType"], dt["T"])[3], 4)
inc_corr = round(compare_data(dt["inc"], dt["INCL"], sigma = True)[3], 3)
a26_corr = round(compare_data(dt["a26_1"], dt["R1"], sigma =False)[3], 3)

|      LCV      |        HECATE        |                        Description                        | Pearson Correlation \[-1,1\] |
|:----------------:|:----------------:|:-----------------:|:----------------:|
|     TType     |   T (with errors)    | Numerical Hubble type following the de Vaucouleurs system |    `{python} ttype_corr`     |
|      inc      |         INCL         |                     Inclination (deg)                     |     `{python} inc_corr`      |
| a26_1 (Major) | R1 (Semi-major axis) |                 angular diameter (arcmin)                 |     `{python} a26_corr`      |

::: panel-tabset
## Galaxy Types


```{=html}
<table style="border-collapse:collapse;border-spacing:0" class="tg"><thead>
<tr><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Hubble stage T </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">−6 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">−5 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">−4 </th>
<th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">−3 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">−2 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">−1 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">0 </th>
<th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">1 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">2 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">3 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">4 </th>
<th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">5 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">6 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">7 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">8 </th>
<th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">9 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">10 </th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">11 </th></tr>
</thead>
<tbody>
<tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">de Vaucouleurs class </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">cE </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">E </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">E+ </td>
<td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">S0− </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">S00 </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">S0+ </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">S0/a </td>
<td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sa </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sab </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sb </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sbc </td>
<td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sc </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Scd </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sd </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sdm </td>
<td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sm </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Im </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal"> </td></tr>
<tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Approximate Hubble class </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal" colspan="3">E </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal" colspan="3">S0 </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">S0/a </td>
<td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sa </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sa-b </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sb </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sb-c </td>
<td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal" colspan="3">Sc </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Sc-Irr </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal">Irr I </td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal"></td>
<td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:center;vertical-align:top;word-break:normal"></td></tr>
</tbody></table>
```

In [ ]:
scatter_plot(dt["TType"], dt["T"], yerr = dt["E_T"])

**Percentage change:**


In [ ]:
dt["diff_T"] = relative_diff(hec = dt["T"], lcv = dt["TType"])
percent_desc_histogram(dt["diff_T"], zoom = [-200,200])

## Inclination


In [ ]:
temp = dt[["inc", "INCL"]].to_pandas()

sns.histplot(temp[["inc", "INCL"]], kde = True)
plt.xlabel("Inclination of galaxies [deg]")
plt.show()

temp["Percentage Change [%]"] = (-temp["inc"] + temp["INCL"])/temp["INCL"]
temp.loc[np.isinf(temp["Percentage Change [%]"]), "Percentage Change [%]"] = np.nan
sns.histplot(temp["Percentage Change [%]"], kde = True)
plt.show()
temp.describe()

We can see that for values in the range $[\sim 30^\circ,\sim 80^\circ]$, the values of the LCV inclination are higher. However, since their means, median, min and maxes are similar and the percentage change is practically 0% (mean, median, $\sigma$ = 0 with a range $[-3\%,1\%]$), we can ignore the differences and assume they are the same values.

## Major Axis


In [ ]:
scatter_plot(dt["a26_1"], dt["R1"])

it is not very clear if we truly have a correlation or not. We need to see the linear correlation of the decimal logarithms.


In [ ]:
dt["log_a26"] = np.log10(dt["a26_1"].data)
dt["log_a26"].unit = u.arcmin
dt["log_R1"] = np.log10(dt["R1"].data)
dt["log_R1"].unit = u.arcmin
dt["log_R1"].info

In [ ]:
scatter_plot(dt["log_a26"], dt["log_R1"], sigma = True)

:::

## Luminosities


In [ ]:
#| echo: false
logKLum_corr = round(compare_data(dt["logKLum"], dt["logL_K"], sigma = True)[3], 3)

|   LCV   | HECATE | Description | Pearson Correlation \[-1,1\] |
|:-------:|:------:|:-----------:|:----------------------------:|
| logKLum | logL_K |             |   `{python} logKLum_corr`    |


In [ ]:
#| echo: false
dt["diff_L_K"] = relative_diff(hec = dt["logL_K"].value, lcv = dt["logKLum"])
temp = dt[["logKLum", "logL_K", "diff_L_K"]].to_pandas()
sns.histplot(temp["diff_L_K"], kde = True)
plt.xlabel("Percentage change [%]")
plt.title("K-band Luminosities")
plt.show()

In [ ]:
temp.rename(columns = {"logKLum":"log(L_K)_{LCV}$", "logL_K":"log(L_K)_{HEC}", "diff_L_K":"Percentage Change [%]"}, inplace=True)

In [ ]:
temp.describe()

## Magnitudes


In [ ]:
#| echo: false
mag_B_corr = round(compare_data(dt["mag_B"], dt["BT"], sigma = True)[3], 3)
Kmag_corr = round(compare_data(dt["Kmag"], dt["K"], sigma = True)[3], 3)

|         LCV         |      HECATE      |         Description         | Pearson Correlation \[-1,1\] |
|:----------------:|:----------------:|:----------------:|:-----------------:|
| mag_B (with errors) | BT (with errors) |                             |    `{python} mag_B_corr`     |
|        Kmag         |        K         | 2MASS band magnitude (both) |     `{python} Kmag_corr`     |

::: panel-tabset
## B mag


In [ ]:
scatter_plot(dt["mag_B"], dt["BT"], yerr = dt["E_BT"], sigma=True)

## K mag


In [ ]:
scatter_plot(dt["Kmag"], dt["K"], yerr = dt["E_K"], sigma = True)

In [ ]:
dt["diff_K_mag"] = relative_diff(dt["Kmag"], dt["K"])

temp = dt[["diff_K_mag"]].to_pandas()
c = temp["diff_K_mag"]
up_clip = c.mean() + 3*c.std()
low_clip = c.mean() - 3*c.std()
temp["Percentage Change, after 3 sigma clip [%]"] = temp["diff_K_mag"].clip(low_clip, up_clip)
temp.rename(columns={"diff_K_mag":"Percentage Change [%]"}, inplace=True)
temp.describe()

In [ ]:
sns.histplot(temp, kde = True)
plt.title("Percentage change Distribution [%]")
plt.xlabel(f"K band magnitude [{dt['K'].unit}]")
plt.show()

:::

## Absorpsion


In [ ]:
#| echo: false
ab_corr = round(compare_data(dt["AB"], dt["AG"])[3], 3)
#ab_int_corr = round(compare_data(dt["AB_int"], dt["AI"])[3], 3)

|  LCV   | HECATE |                   Description                   | Pearson Correlation \[-1,1\] |
|:----------------:|:----------------:|:-----------------:|:----------------:|
|   AB   |   AG   |    Galactic extinction/absorption in B band     |      `{python} ab_corr`      |
| AB_int |   AI   | Internal/Intrinsic B band extinction/absorption |              0               |

::: panel-tabset
## Galactic


In [ ]:
scatter_plot(dt["AB"], dt["AG"], logscale = True)#, scatter_plot(dt["AB_int"], dt["AI"])

## Internal


In [ ]:
plt.scatter(dt["AB_int"],dt["AI"],)
plt.xlabel(f"LCV, AB_int [{dt['AB_int'].unit}]")
plt.ylabel(f"HECATE, AI [{dt['AI'].unit}]")
plt.yscale("log")
plt.show()

In [ ]:
sns.jointplot(x=dt["AB_int"], y=dt["AI"], kind="reg",)

:::

## SFR


In [ ]:
#| echo: false
# Extract the relevant SFR columns
sfr_columns = ["logSFR_TIR", "logSFR_FIR", "logSFR_60u", "logSFR_12u", "logSFR_22u", "logSFR_HEC", "logSFR_GSW", "logSFRFUV", "logSFRHa"]
sfr_data = dt[sfr_columns].to_pandas()

# Count the number of non-NaN cells for each column
non_nan_counts = sfr_data.notna().sum()

|  LCV   |   HECATE   |                           Description                            |                  Count                  |
|:----------------:|:----------------:|:-----------------:|:----------------:|
|        | logSFR_TIR | Decimal logarithm of the total-infrared SFR estimate \[Msol/yr\] | `{python} non_nan_counts["logSFR_TIR"]` |
|        | logSFR_FIR |  Decimal logarithm of the far-infrared SFR estimate \[Msol/yr\]  | `{python} non_nan_counts["logSFR_FIR"]` |
|        | logSFR_60u |      Decimal logarithm of the 60um SFR estimate \[Msol/yr\]      | `{python} non_nan_counts["logSFR_60u"]` |
|        | logSFR_12u |      Decimal logarithm of the 12um SFR estimate \[Msol/yr\]      | `{python} non_nan_counts["logSFR_12u"]` |
|        | logSFR_22u |      Decimal logarithm of the 22um SFR estimate \[Msol/yr\]      | `{python} non_nan_counts["logSFR_22u"]` |
|        | logSFR_HEC |  Decimal logarithm of the homogenised SFR estimate \[Msol/yr\]   | `{python} non_nan_counts["logSFR_HEC"]` |
|        | logSFR_GSW |       Decimal logarithm of the SFR in GSWLC-2 \[Msol/yr\]        | `{python} non_nan_counts["logSFR_GSW"]` |
| SFRFUV |            |             FUV derived integral star formation rate             | `{python} non_nan_counts["logSFRFUV"]`  |
| SFRHa  |            |          H{alpha} derived integral star formation rate           |  `{python} non_nan_counts["logSFRHa"]`  |

::: panel-tabset
## Heatmap


In [ ]:
#| echo: false
sfr_data.drop("logSFR_GSW", axis=1, inplace=True)
# Compute the correlation matrix
corr_matrix = sfr_data.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Heatmap of SFR Values")
plt.show()

## Scatter Grid


In [ ]:
# Create a pair grid 
sns.pairplot(sfr_data, corner=True)
plt.show()

## Density Grids


In [ ]:
# Create a pair grid 
sns.pairplot(sfr_data, corner=True, kind="kde", diag_kind="hist")
plt.show()

:::

It is possible we dont see a good correlation because we dont have a big enough common sample

## Masses


In [ ]:
#| echo: false
# Extract the relevant mass columns
mass_columns = ["logM26", "logMHI", "logM_HEC", "logM_GSW", "logStellarMass"]
mass_data = dt[mass_columns].to_pandas()

# Count the number of non-NaN cells for each column
non_nan_counts = mass_data.notna().sum()

|      LCV       |  HECATE  |                        Description                        |                    Count                    |
|:----------------:|:----------------:|:----------------:|:----------------:|
|     logM26     |          |              Log mass within Holmberg radius              |     `{python} non_nan_counts["logM26"]`     |
|     logMHI     |          |              Log mass within Holmberg radius              |     `{python} non_nan_counts["logMHI"]`     |
|                | logM_HEC |      Decimal logarithm of the stellar mass \[Msol\]       |    `{python} non_nan_counts["logM_HEC"]`    |
|                | logM_GSW | Decimal logarithm of the stellar mass in GSWLC-2 \[Msol\] |    `{python} non_nan_counts["logM_GSW"]`    |
| logStellarMass |          |               Stellar Mass from M\_\*/L=0.6               | `{python} non_nan_counts["logStellarMass"]` |

::: panel-tabset
## Stellar Masses Comparison


In [ ]:
dt["diff_M"] =  relative_diff(hec = dt["logM_HEC"]._convert_data_to_col, lcv = dt["logStellarMass"])

## Heatmap


In [ ]:
mass_data.drop("logM_GSW", axis=1, inplace=True)
# Compute the correlation matrix
mass_corr_matrix = mass_data.corr()

# Create a heatmap
sns.heatmap(mass_corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Heatmap of Mass Values")
plt.show()

## Plot Grid


In [ ]:
# Create a pair grid
sns.pairplot(mass_data, corner=True)
plt.show()

## Density Grid


In [ ]:
# Create a pair grid
sns.pairplot(mass_data, corner=True, kind="kde", diag_kind="hist")
plt.show()

:::